# Canned Example: sign(x)
This notebook defines our canned example which defines several labeling functions that model the sign function. (See Overleaf document for full definition of this example.)

In [1]:
import numpy as np
from snorkel.labeling import labeling_function, LFApplier, LFAnalysis
from snorkel.labeling.model.label_model import LabelModel

In [2]:
# Parameters for the analysis
N_LFS = 10
N_DPS = 10000

SEED_RNG = False
SEED = 12345

In [3]:
# Defining labels for sake of clarity
ABSTAIN = 0
NEGATIVE = -1
POSITIVE = 1

In [4]:
# Seed numpy.random for reproducible results
if SEED_RNG:
    np.random.seed(SEED)

## Define the Goal Model

In [5]:
def f_star(x):
    return 1 if x == 0 else np.sign(x)

## Define the Dataset

In [6]:
X_data = np.random.uniform(-1, 1, N_DPS)

f_star_vectorized = np.vectorize(f_star)
Y_data = f_star_vectorized(X_data)

## Define Labeling Functions

In [7]:
def f(p_f, p_A, x):    
    # Draw from Bernoulli distribution
    z_f = np.random.binomial(1, p_f, 1)[0]
    z_A = np.random.binomial(1, p_A, 1)[0]
    
    return int( (f_star(x) * z_f - f_star(x) * (1 - z_f)) * (1 - z_A) )

In [8]:
lfs = []

# Generate several labeling functions for different values of p_f and p_A
for i in range(N_LFS):
    p_f = np.random.rand(1)[0]
    p_A = np.random.rand(1)[0]
    
    @labeling_function(name="lf{}({:.4f},{:.4f})".format(i, p_f, p_A))
    def lf(x):
        return f(p_f, p_A, x)
    
    lfs.append(lf)

## Apply Labeling Functions to the Data

In [9]:
applier = LFApplier(lfs)
L_data = applier.apply(X_data, progress_bar=True)

10000it [00:01, 5763.10it/s]


In [10]:
LFAnalysis(L_data, lfs=lfs).lf_summary(Y_data)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
"lf0(0.8646,0.1559)",0,"[0, 1]",0.5372,0.5257,0.2868,3843,344,0.715376
"lf1(0.4703,0.9647)",1,"[0, 1]",0.5342,0.5228,0.2822,3866,352,0.723699
"lf2(0.0026,0.4366)",2,"[0, 1]",0.5351,0.5219,0.2854,3862,359,0.721734
"lf3(0.2596,0.0961)",3,"[0, 1]",0.5402,0.5297,0.2897,3876,334,0.717512
"lf4(0.7671,0.2620)",4,"[0, 1]",0.5349,0.5238,0.2832,3857,353,0.721069
"lf5(0.3639,0.4487)",5,"[0, 1]",0.5290,0.5180,0.2802,3825,346,0.723062
"lf6(0.0936,0.6155)",6,"[0, 1]",0.5341,0.5217,0.2815,3826,361,0.716345
"lf7(0.9997,0.9435)",7,"[0, 1]",0.5344,0.5263,0.2839,3874,347,0.724925
"lf8(0.1725,0.5426)",8,"[0, 1]",0.5287,0.5171,0.2810,3816,330,0.721770
"lf9(0.8272,0.0688)",9,"[0, 1]",0.5386,0.5269,0.2861,3849,371,0.714631


## Experiments